# Text Generation

In [4]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.utils as ku 

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

## Step 1 : Get the Corpus

In [5]:
path = tf.keras.utils.get_file('sonnets.txt', 
                               'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt')
print (path)
    
data = open(path).read()

corpus = data.lower().split("\n")

98304/93578 [===============================] - 0s 1us/step
C:\Users\Amir\.keras\datasets\sonnets.txt


## Step 2 : Tokenize the Corpus

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
total_words

3211

## Step 3 : Create n-Gram

In [8]:
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

## Step 4 : Pad Sequences

In [9]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, 
                                         maxlen=max_sequence_len, 
                                         padding='pre'))

## Step 5 : X and y - Values

In [10]:
# create (predictors and label) X and y

# label one hot encoded
X = input_sequences[:,:-1]
y_label = input_sequences[:,-1]

y = tf.keras.utils.to_categorical(y_label, num_classes = total_words)

## Step 6 : Create Model

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))

model.add(Dense(total_words/2, activation='relu', 
                kernel_regularizer=regularizers.l2(0.01)))

model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

print(model.summary())

history = model.fit(X, y, epochs=100, verbose=1)
    
import matplotlib.pyplot as plt
acc = history.history['accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

## Step 7 : Generate Text

In [ ]:
seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list), axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)